# Override Result with Middleware

This notebook demonstrates how middleware can modify or override function results.

## Key Concepts

- **Result Transformation**: Modify function outputs before returning
- **Result Enrichment**: Add metadata or additional information
- **Result Validation**: Ensure outputs meet quality standards

## Prerequisites

- Azure OpenAI endpoint and deployment configured in `.env`
- `agent-framework` package installed
- Azure CLI authentication

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../.env')

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"Project Endpoint: {project_endpoint}")
print(f"Model Deployment: {model_deployment_name}")

In [ ]:
from collections.abc import AsyncIterable, Awaitable, Callable
from random import randint
from typing import Annotated

from agent_framework import (
    AgentRunContext,
    AgentRunResponse,
    AgentRunResponseUpdate,
    ChatMessage,
    Role,
    TextContent,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

## Define Weather Tool Function

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

## Result Override Middleware

This middleware demonstrates how to:
- Execute the original function first
- Override results for both streaming and non-streaming responses
- Use `context.is_streaming` to detect the execution mode
- Create custom async generators for streaming overrides

In [ ]:
async def weather_override_middleware(
    context: AgentRunContext, next: Callable[[AgentRunContext], Awaitable[None]]
) -> None:
    """Middleware that overrides weather results for both streaming and non-streaming cases."""

    # Let the original agent execution complete first
    await next(context)

    # Check if there's a result to override (agent called weather function)
    if context.result is not None:
        # Create custom weather message
        chunks = [
            "Weather Advisory - ",
            "due to special atmospheric conditions, ",
            "all locations are experiencing perfect weather today! ",
            "Temperature is a comfortable 22°C with gentle breezes. ",
            "Perfect day for outdoor activities!",
        ]

        if context.is_streaming:
            # For streaming: create an async generator that yields chunks
            async def override_stream() -> AsyncIterable[AgentRunResponseUpdate]:
                for chunk in chunks:
                    yield AgentRunResponseUpdate(contents=[TextContent(text=chunk)])

            context.result = override_stream()
        else:
            # For non-streaming: just replace with the string message
            custom_message = "".join(chunks)
            context.result = AgentRunResponse(messages=[ChatMessage(role=Role.ASSISTANT, text=custom_message)])

## Example: Non-streaming Result Override

In [ ]:
async def non_streaming_example():
    """Example demonstrating result override with middleware for non-streaming."""
    print("--- Non-streaming Example ---")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather assistant. Use the weather tool to get current conditions.",
            tools=get_weather,
            middleware=weather_override_middleware,
        ) as agent,
    ):
        query = "What's the weather like in Seattle?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await non_streaming_example()

## Example: Streaming Result Override

In [ ]:
async def streaming_example():
    """Example demonstrating result override with middleware for streaming."""
    print("\n--- Streaming Example ---")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather assistant. Use the weather tool to get current conditions.",
            tools=get_weather,
            middleware=weather_override_middleware,
        ) as agent,
    ):
        query = "What's the weather like in Portland?"
        print(f"User: {query}")
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print()  # New line after streaming

await streaming_example()

## Key Takeaways

- Middleware can override agent results after execution using `context.result`
- Execute the original function first with `await next(context)`, then override
- Use `context.is_streaming` to detect streaming vs non-streaming execution
- For streaming: create async generator that yields `AgentRunResponseUpdate` objects
- For non-streaming: replace with `AgentRunResponse` containing custom messages
- Useful for result filtering, formatting, enhancement, or providing alternative responses

## Next Steps

- **Shared state** (notebook 9) for cross-middleware communication